In [ ]:
# STEP 1: Install dependencies
!pip install openpyxl statsmodels

# STEP 2: Import libraries
import pandas as pd
import numpy as np
import scipy.stats as stats
from statsmodels.stats.inter_rater import fleiss_kappa
from openpyxl.utils import column_index_from_string

# STEP 3: Load Excel file (replace with your path if needed)
file_path = "/content/sample_data/experts_clean_COMBINED.xlsx"
df = pd.read_excel(file_path, sheet_name="Sheet1")

# STEP 4: Isolate binary concept columns (Z to HM), excluding specific metadata
start_col = column_index_from_string('Z') - 1
end_col = column_index_from_string('HM') - 1
excluded_cols = ['BE','BF','CL','CM','DS','DT','EZ','FA','GG','GH','HN','HO','HP']
excluded_indices = [column_index_from_string(c) - 1 for c in excluded_cols]

concept_cols = [
    df.columns[i] for i in range(start_col, end_col + 1)
    if i not in excluded_indices and df[df.columns[i]].dropna().isin([0, 1]).all()
]

# STEP 5: Optional confidence/struggle columns
confidence_cols = [col for col in df.columns if 'confidence' in col.lower()]
struggle_cols = [col for col in df.columns if 'struggle' in col.lower()]

# STEP 6: Compute agreement rates
agreement_rates = df[concept_cols].mean().sort_values(ascending=False)

# STEP 7: Build Fleiss matrix from complete rows only
df_clean = df[concept_cols].dropna(axis=0, how='any')
n_raters = df_clean.shape[0]

fleiss_matrix = []
for col in df_clean.columns:
    counts = df_clean[col].value_counts().reindex([0, 1], fill_value=0)
    fleiss_matrix.append([counts[0], counts[1]])
fleiss_matrix = np.array(fleiss_matrix)

# STEP 8: Calculate Fleiss’ Kappa
kappa = fleiss_kappa(fleiss_matrix)

# STEP 9: Compute CI, Z-score, p-value
def fleiss_kappa_ci(kappa_val, n_raters, n_items, alpha=0.05):
    se_kappa = np.sqrt((2 * (1 - kappa_val)) / (n_items * n_raters * (n_raters - 1)))
    z = kappa_val / se_kappa
    p_value = 2 * (1 - stats.norm.cdf(abs(z)))
    ci_half = stats.norm.ppf(1 - alpha / 2) * se_kappa
    return se_kappa, z, p_value, (max(-1.0, kappa_val - ci_half), min(1.0, kappa_val + ci_half))

se, z, p, ci = fleiss_kappa_ci(kappa, n_raters, len(concept_cols))
ci_lower, ci_upper = float(ci[0]), float(ci[1])

# STEP 10: Print full results
print("===================================")
print("     Fleiss' Kappa Summary (AUTO)  ")
print("===================================")
print(f"✔️  Fleiss' Kappa Value     : {kappa:.4f}")
print(f"📊  Z-Score                : {z:.4f}")
p_str = f"{p:.2e} (p < 0.0001)" if p < 1e-4 else f"{p:.4f}"
print(f"📈  p-Value (Z-test)       : {p_str}")
print(f"🔒  95% Confidence Interval: ({ci_lower:.4f}, {ci_upper:.4f})")
print("-----------------------------------")
print(f"📌 Interpretation          : {'Statistically significant' if p < 0.05 else 'Not statistically significant'}")

# STEP 11: Show top 10 agreement rates
print("\n✅ Top 10 Agreement Rates:")
print(agreement_rates.head(10))

# STEP 12: Save agreement rates to clean CSV
agreement_rates.to_csv("agreement_rates.csv", float_format="%.4f", index_label="Concept", header=["AgreementRate"])

     Fleiss' Kappa Summary (AUTO)  
✔️  Fleiss' Kappa Value     : 0.1357
📊  Z-Score                : 31.5019
📈  p-Value (Z-test)       : 0.00e+00 (p < 0.0001)
🔒  95% Confidence Interval: (0.1273, 0.1442)
-----------------------------------
📌 Interpretation          : Statistically significant

✅ Top 10 Agreement Rates:
Wire         0.826087
Dog          0.814815
Mouse        0.814815
Cat          0.814815
Monitor      0.814815
Keyboard     0.814815
Saxophone    0.782609
Printer      0.777778
Eagle        0.740741
Penguin      0.740741
dtype: float64
